In [ ]:
%matplotlib inline

In [ ]:
!conda install -y -c conda-forge zarr

Adapted in part from Chase et al. (2023): https://github.com/ai2es/WAF_ML_Tutorial_Part2/blob/main/jupyter_notebooks/Notebook07_CNNs.ipynb

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
import zarr
import numpy as np

In [ ]:
#base_folder ='/share/share/sub-sevir/'
base_folder = '/Users/seanfreeman/Documents/Teaching/Spring_2024/'

In [ ]:
training_ds = xr.open_dataset(base_folder+"/sub-sevir/sub-sevir-train.zarr", engine='zarr')

In [ ]:
training_ds

In [ ]:
np.unique(training_ds['label_1d_class'])
#0: not thunderstorm, 1: thunderstorm

In [ ]:
ds_train = tf.data.Dataset.from_tensor_slices((training_ds.features.values,training_ds.label_1d_class.values))
ds_train = ds_train.shuffle(ds_train.cardinality().numpy())
batch_size = 32
ds_train = ds_train.batch(batch_size)


In [ ]:
ds_train

In [ ]:
tf.keras.backend.clear_session()
keras.utils.set_random_seed(1)
input1 = keras.layers.Input(shape=(48,48,4))
input_pool = keras.layers.MaxPool2D(pool_size=(2, 2), padding='same')(input1)
conv_layer1 = keras.layers.Conv2D(1, kernel_size=3, activation='relu')(input_pool)
flat_layer = keras.layers.Flatten()(conv_layer1)
out_layer = keras.layers.Dense(1, activation='sigmoid')(flat_layer)
model_pool = keras.models.Model(inputs=[input1], outputs=[out_layer])
model_pool.compile(optimizer=keras.optimizers.legacy.Adam(), loss='mean_squared_error')
model_pool.summary()


In [ ]:
# predict before training on random weights
out_pool = model_pool.predict(training_ds.features.values)

In [ ]:
out_pool.shape

In [ ]:
plt.hist(out_pool)

In [ ]:
model_pool.fit(ds_train, epochs=3)

In [ ]:
# convolution weights
filters = model_pool.weights[0]
filters.shape


In [ ]:
fig,axes = plt.subplots(1,4,figsize=(20,5))

for i in np.arange(0,4):
    pm = axes[i].imshow(filters[:,:,i,0],vmin=-1,vmax=1,cmap='seismic')
    plt.colorbar(pm,ax=axes[i])


In [ ]:
for l,layer in enumerate(model_pool.layers):
    print(l,layer.name)
    print(l, layer.weights)
